In [1]:
import os
os.chdir('/workspace/FutureGPT2/src/')

import numpy as np
from torch import optim, nn, Tensor
from torch.nn import functional as F
import torch
import wandb
from transformers import GPT2Config, GPT2Model
import transformers
import lightning as L
from inspect import signature, _ParameterKind
import copy
import gc
import datasets
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
from lightning.pytorch.callbacks import ModelCheckpoint, LearningRateMonitor
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from itertools import repeat

from models.regression_model import *
from data.synthetic import *
from models.myopic_model import to_myopic_gpt2

%load_ext autoreload
%autoreload 2

In [2]:
if torch.cuda.get_device_capability()[0] >= 8:
    torch.set_float32_matmul_precision('high')

In [3]:
fixed_data_params = {
    'seq_len': 64,
}
free_data_params = {
    'conv': 10,
    'shift': 1,
    'p': 1.,
}
data_params = {**fixed_data_params, **free_data_params}

fixed_model_params = {
    'n_positions': 64,
    'in_dim': 2,
    'n_inner': None,
    'attn_pdrop': 0.,
    'embd_pdrop': 0.,
    'resid_pdrop': 0.,
}
free_model_params = {
    'lr': 1e-4,
    'n_embd': 256,
    'n_head': 4,
    'n_layer': 2,
    'activation_function': 'relu',
    'decay': 0.1,
}
model_params = {**fixed_model_params, **free_model_params}

In [4]:
wandb.login(key='os.environ[WANDB_API_KEY]', relogin=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/wwu/.netrc


True

In [5]:
#val = DataLoader(SyntheticSeqDataset(size=10000, **data_params), batch_size=512, num_workers=95)

In [ ]:
train = DataLoader(SyntheticSeqDataset(size=100_000_000, **data_params), batch_size=512, num_workers=95)
#train = DataLoader(SyntheticSeqDataset(size=10_000, **data_params), batch_size=512, num_workers=95)
BETA=0.0
NAME = '_'.join(
    [f'SYNTH-GPT2-COS'] + 
    [f'{k}-{v}' for k, v in {**free_data_params, **free_model_params}.items()]
)
PROJ = 'LAISR_FUTURE_SYNTHETIC'
wandb_logger = WandbLogger(
    name=NAME,
    project=PROJ,
    log_model=False,   # Only save checkpoints locally
)
lr_monitor = LearningRateMonitor()
checkpoint_callback = ModelCheckpoint(
    dirpath="/workspace/checkpoints",
    filename=NAME + "_{global_step}_{train_loss:.2f}",
    every_n_epochs=1,
    save_top_k=1,
    monitor='train_loss',
    mode='min',
)
early_stop_callback = EarlyStopping(
    monitor='train_loss',
    divergence_threshold=10000,
    min_delta=0.00,
    patience=100000,
    verbose=False,
    mode='min',
)
trainer = L.Trainer(
    fast_dev_run=False,
    logger=wandb_logger,
    val_check_interval=1.0,
    #check_val_every_n_epoch=5,
    callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
    max_epochs=1,
    enable_progress_bar=True,
)
model = LitGPT2RegModel(**model_params)
wandb_logger.watch(model.model, log='all')
# model = to_myopic_gpt2(
#     model,
#     [None for _ in range(12)],
#     beta=BETA,
# )
trainer.fit(
    model=model,
    train_dataloaders=train,
    #val_dataloaders=val,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..
wandb: Currently logged in as: wilswu. Use `wandb login --relogin` to force relogin


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/home/wwu/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /workspace/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name    | Type      | Params
--------------------------------------
0 | model   | GPT2Model | 1.6 M 
1 | loss_fn | HuberLoss | 0     
--------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.390     Total estimated model params size (MB)


NUM TRAINING STEPS 195313


Training: |          | 0/? [00:00<?, ?it/s]